## Gathering Data for Analysis
This project will be based on data from the following sources:
a) Foursquare, which will provide location data for the southeastern Manitoba region and amenities located within it, and b) Trail Forks, which identifies the most established mountain biking trails in a region

This project will take the geographic locations of popular cycling routes from Trail Forks as one dataset. It will take the geographic locations of trails from Trail Forks as another dataset. It will plot both on a map and show their location relative to area hotels. 

The results will provide the event organizer with a set of possible venues for a mountain bike race in southeastern Manitoba. From there, the organizer can consider other factors, such as proximity of restaurants that could potentially cater a post-race meal, or features of the possible trails (such as length of the trail or nature of the terrain) that could further inform the decision-making process to select an event venue.

In [73]:
#Let's start by importing the Python libraries we'll need to complete this assignment

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [74]:
#Next, we'll upload the Client ID and Client Secret(essentially, a password) so we can connect to Foursquare

CLIENT_ID = 'KOYZEXK0ES5ID3C1WLGKIHXJBWRIS4MYQJU2SKEFS333X5DQ' # your Foursquare ID
CLIENT_SECRET = 'KHRSIOGE3FCDQ1IN2BSPIBKKAO1EH2XU3G1LIKWO3JTPG0X0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KOYZEXK0ES5ID3C1WLGKIHXJBWRIS4MYQJU2SKEFS333X5DQ
CLIENT_SECRET:KHRSIOGE3FCDQ1IN2BSPIBKKAO1EH2XU3G1LIKWO3JTPG0X0


In [75]:
#Let's use Steinbach, Manitoba, Canada, as the initial location for our analysis, letting it act as a reference point

address = 'Steinbach, Manitoba, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

49.530557 -96.69342


In [62]:
#Next, let's use hotel venues as the focus for our search query, and use a 200 kilometre radius to return lots of results

search_query='hotel'
radius=200000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=KOYZEXK0ES5ID3C1WLGKIHXJBWRIS4MYQJU2SKEFS333X5DQ&client_secret=KHRSIOGE3FCDQ1IN2BSPIBKKAO1EH2XU3G1LIKWO3JTPG0X0&ll=49.530557,-96.69342&v=20180604&query=hotel&radius=200000&limit=30'

In [63]:
#Let's take a look at the results of the above query

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f00e290889e0936195fbbe1'},
 'response': {'venues': [{'id': '4f6c709fbb3d8e65c21090e4',
    'name': 'La Broquerie Hotel',
    'location': {'address': '3 Pignons St',
     'lat': 49.517595,
     'lng': -96.502317,
     'labeledLatLngs': [{'label': 'display',
       'lat': 49.517595,
       'lng': -96.502317}],
     'distance': 13884,
     'postalCode': 'R0A 0W0',
     'cc': 'CA',
     'city': 'La Broquerie',
     'state': 'MB',
     'country': 'Canada',
     'formattedAddress': ['3 Pignons St',
      'La Broquerie MB R0A 0W0',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1fa931735',
      'name': 'Hotel',
      'pluralName': 'Hotels',
      'shortName': 'Hotel',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1593893461',
    'hasPerk': False},
   {'id': '4e2361e6e4cdf68591996eb5',
    'name': 'St Anne Hotel Chinese Restaurant',
   

In [76]:
#Now let's create a dataframe based on the information above that was produced from the Foursquare query

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head(20)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,4f6c709fbb3d8e65c21090e4,La Broquerie Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1593893461,False,3 Pignons St,49.517595,-96.502317,"[{'label': 'display', 'lat': 49.517595, 'lng':...",13884,R0A 0W0,CA,La Broquerie,MB,Canada,"[3 Pignons St, La Broquerie MB R0A 0W0, Canada]",NaN,NaN,NaN
1,4e2361e6e4cdf68591996eb5,St Anne Hotel Chinese Restaurant,[],v-1593893461,False,NaN,49.672748,-96.657028,"[{'label': 'display', 'lat': 49.67274811812094...",16044,NaN,CA,St. Anne,MB,Canada,"[St. Anne MB, Canada]",NaN,NaN,NaN
2,4d8239892fb73704a4238811,The Alouette Hotel,[],v-1593893461,False,101 2nd St. E,49.673719,-96.822029,"[{'label': 'display', 'lat': 49.673719, 'lng':...",18440,R0A 0X0,CA,The Pas,MB,Canada,"[101 2nd St. E, The Pas MB R0A 0X0, Canada]",NaN,NaN,NaN
3,50fb1e1ee4b045129eff1d63,St. Malo Hotel,"[{'id': '4bf58dd8d48988d118941735', 'name': 'D...",v-1593893461,False,NaN,49.315830,-96.954730,"[{'label': 'display', 'lat': 49.31583, 'lng': ...",30485,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN,NaN
4,56254264498ebd5d2e1b8916,Ste Anne Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1593893461,False,103 Dawson Rd,49.657334,-96.455816,"[{'label': 'display', 'lat': 49.6573337, 'lng'...",22206,R5H 1B6,CA,Ste. Anne,MB,Canada,"[103 Dawson Rd, Ste. Anne MB R5H 1B6, Canada]",NaN,NaN,NaN
5,5089bebfe4b03c5c38cf257e,Richer Inn Motor Hotel,"[{'id': '5bae9231bedf3950379f89cb', 'name': 'I...",v-1593893461,False,Hwy #1,49.661689,-96.463006,"[{'label': 'display', 'lat': 49.66168879691342...",22124,R0E 1S0,CA,Richer,MB,Canada,"[Hwy #1 (At Hwy #302), Richer MB R0E 1S0, Canada]",At Hwy #302,NaN,NaN
6,5220131e11d28b6cc5e1b8e0,Grande Allen Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1593893461,False,601 grande all east,49.774100,-96.707700,"[{'label': 'display', 'lat': 49.7741, 'lng': -...",27130,NaN,CA,Quebec,Quebec,Canada,"[601 grande all east, Quebec Quebec, Canada]",NaN,NaN,NaN
7,4d1bf509ffe98eec30ac976c,Ile Des Chenes Motor Hotel - Wrangler's Bar,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1593893461,False,654 Lacroix St.,49.713447,-96.994623,"[{'label': 'display', 'lat': 49.71344683026852...",29770,R0A 0T0,CA,Ile Des Chenes,MB,Canada,"[654 Lacroix St. (Old Highway 59), Ile Des Che...",Old Highway 59,NaN,NaN
8,4d0d98f8903d37040b56c555,St. Adolphe Motor Hotel,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1593893461,False,NaN,49.671186,-97.111279,"[{'label': 'display', 'lat': 49.67118556225096...",33969,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN,NaN
9,4bf3166ec440c9b63f8b0104,Curtis Gordon Motor Hotel,"[{'id': '4bf58dd8d48988d1fb931735', 'name': 'M...",v-1593893461,False,1011 Henderson Hwy.,49.935308,-97.094883,"[{'label': 'display', 'lat': 49.935307768905, ...",53520,R2K 2M2,CA,Winnipeg,MB,Canada,[1011 Henderson Hwy. (bwtn Leighton Ave. & McL...,bwtn Leighton Ave. & McLeod Ave.,383913260,NaN


In [77]:
#Next up, let's create a dataframe containing the desired information from TrailForks

data = {'Place_Name':['Sandilands', 'St. Malo', 'Granite Groove Out', 'McGillivray Falls', 'Falcon Lake Trans Canada Trail', 'Falcon Ridge Ski Slopes', 'Crystal Springs Colony'],
        'Latitude': [49.3246, 49.3141, 50.1503, 49.8095, 49.7030, 49.690300,  49.546950],
        'Longitude': [-96.2932, -96.9536, -95.8798, -95.2379, -95.2433, -95.317180, -97.116850],
        'Green_circle': [8, 4, 3, 3, 13, 22, 2],
        'Blue_square': [12, 0, 4, 18, 0, 21, 4],
        'Black_diamond': [1, 0, 0, 0, 0, 2, 0],
        'Total_distance': [16, 5.2, 7.2, 31, 28, 32, 2.5]}

dfTrailForks = pd.DataFrame(data,columns=['Place_Name', 'Latitude', 'Longitude', 'Green_circle', 'Blue_square', 'Black_diamond', 'Total_distance'])



In [78]:
dfTrailForks.head(7)

,Place_Name,Latitude,Longitude,Green_circle,Blue_square,Black_diamond,Total_distance
0,Sandilands,49.32460,-96.29320,8,12,1,16.0
1,St. Malo,49.31410,-96.95360,4,0,0,5.2
2,Granite Groove Out,50.15030,-95.87980,3,4,0,7.2
3,McGillivray Falls,49.80950,-95.23790,3,18,0,31.0
4,Falcon Lake Trans Canada Trail,49.70300,-95.24330,13,0,0,28.0
5,Falcon Ridge Ski Slopes,49.69030,-95.31718,22,21,2,32.0
6,Crystal Springs Colony,49.54695,-97.11685,2,4,0,2.5


In [79]:
#Before we start working with the Foursquare data in earnest, let's clean up the dataframe a little

dataframe_clean = dataframe
dataframe_clean.columns = [column.split('.')[-1] for column in dataframe_clean.columns]
dataframe_clean.head()


,id,name,categories,referralId,hasPerk,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id,neighborhood
0,4f6c709fbb3d8e65c21090e4,La Broquerie Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1593893461,False,3 Pignons St,49.517595,-96.502317,"[{'label': 'display', 'lat': 49.517595, 'lng':...",13884,R0A 0W0,CA,La Broquerie,MB,Canada,"[3 Pignons St, La Broquerie MB R0A 0W0, Canada]",NaN,NaN,NaN
1,4e2361e6e4cdf68591996eb5,St Anne Hotel Chinese Restaurant,[],v-1593893461,False,NaN,49.672748,-96.657028,"[{'label': 'display', 'lat': 49.67274811812094...",16044,NaN,CA,St. Anne,MB,Canada,"[St. Anne MB, Canada]",NaN,NaN,NaN
2,4d8239892fb73704a4238811,The Alouette Hotel,[],v-1593893461,False,101 2nd St. E,49.673719,-96.822029,"[{'label': 'display', 'lat': 49.673719, 'lng':...",18440,R0A 0X0,CA,The Pas,MB,Canada,"[101 2nd St. E, The Pas MB R0A 0X0, Canada]",NaN,NaN,NaN
3,50fb1e1ee4b045129eff1d63,St. Malo Hotel,"[{'id': '4bf58dd8d48988d118941735', 'name': 'D...",v-1593893461,False,NaN,49.315830,-96.954730,"[{'label': 'display', 'lat': 49.31583, 'lng': ...",30485,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN,NaN
4,56254264498ebd5d2e1b8916,Ste Anne Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1593893461,False,103 Dawson Rd,49.657334,-96.455816,"[{'label': 'display', 'lat': 49.6573337, 'lng'...",22206,R5H 1B6,CA,Ste. Anne,MB,Canada,"[103 Dawson Rd, Ste. Anne MB R5H 1B6, Canada]",NaN,NaN,NaN


In [80]:
#Okay, using some geographic coordinates just outside of Steinbach, Manitoba, as our anchoring point for the map, let's produce our initial map

latitude2 = 49.3332
longitude2 = -96.0361
venues_map = folium.Map(location=[latitude2, longitude2], zoom_start=9) # generate map centred around Steinbach
venues_map



In [81]:
#Let's add some more layers of data to the map above
# add the hotels as blue circle markers
for lat, lng in zip(dataframe_clean.lat, dataframe_clean.lng):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

#add the trails as red circle markers
for Latitude, Longitude in zip(dfTrailForks.Latitude, dfTrailForks.Longitude):
    folium.features.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        color='red',
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)
    
    # display map
venues_map

In [82]:
#Now let's re-create the venues map with the trail networks colour-coded based on whether the total distance of trails available is longer than 15 kilometres
data2 = {'Place_Name':['Sandilands', 'McGillivray Falls', 'Falcon Lake Trans Canada Trail', 'Falcon Ridge Ski Slopes'],
        'Latitude': [49.3246, 49.8095, 49.7030, 49.690300],
        'Longitude': [-96.2932, -95.2379, -95.2433, -95.317180],
        'Green_circle': [8, 3, 13, 22],
        'Blue_square': [12, 18, 0, 21],
        'Black_diamond': [1, 0, 0, 2],
        'Total_distance': [16, 31, 28, 32]}

dfTrailForksLong = pd.DataFrame(data2,columns=['Place_Name', 'Latitude', 'Longitude', 'Green_circle', 'Blue_square', 'Black_diamond', 'Total_distance'])

data3 = {'Place_Name':['St. Malo', 'Granite Groove Out', 'Crystal Springs Colony'],
        'Latitude': [49.3141, 50.1503, 49.546950],
        'Longitude': [-96.9536, -95.8798, -97.116850],
        'Green_circle': [4, 3, 2],
        'Blue_square': [0, 4, 4],
        'Black_diamond': [0, 0, 0],
        'Total_distance': [5.2, 7.2, 2.5]}

dfTrailForksShort = pd.DataFrame(data3,columns=['Place_Name', 'Latitude', 'Longitude', 'Green_circle', 'Blue_square', 'Black_diamond', 'Total_distance'])



In [83]:
#Time to generate a new map that will now show the mountain bike trails in two groups (short and long)
latitude2 = 49.3332
longitude2 = -96.0361
venues_mapFinal = folium.Map(location=[latitude2, longitude2], zoom_start=9) # generate map centred around Steinbach

# add the hotels as blue circle markers
for lat, lng in zip(dataframe_clean.lat, dataframe_clean.lng):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_mapFinal)

#add the trails that are longer than 15km as red circle markers
for Latitude, Longitude in zip(dfTrailForksLong.Latitude, dfTrailForksLong.Longitude):
    folium.features.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        color='red',
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_mapFinal)
    
#add the trails that are shorter than 15km as yellow circle markers
for Latitude, Longitude in zip(dfTrailForksShort.Latitude, dfTrailForksShort.Longitude):
    folium.features.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        color='yellow',
        fill = True,
        fill_color='yellow',
        fill_opacity=0.6
    ).add_to(venues_mapFinal)
    
     # display map
venues_mapFinal

(30, 19)